In [2]:
# 4.1 객체 계획 및 정의

"""객체 모델을 잘 설정/정의해야 한다!(책 참조)"""

# 4.2 다양한 웹사이트 레이아웃 다루기

"""
구조에 대한 지식 없이도 다양한 웹 사이트에서 관련성 있고 유용한 데이터를 추출하는 것은 어렵다.
그렇다면 크롤러가 여러 곳에서 데이터를 수집하여 비교 분석하기 위해서는 반드시
복잡한 알고리즘이나 머신러닝을 사용해야하는 걸까?

그렇지 않다.
다행스럽게도, 본 적도 들어본 적도 없는 사이트에서 데이터를 수집하는 경우는 별로 없다.
대부분 만드는 사람이 미리 선택한 웹사이트 몇 개 또는 몇십 개에서 수집하는 경우가 대부분이다.

따라서 이번에는 두 개의 뉴스 웹사이트를 미리 지정해놓고, 
두 개의 웹사이트에서 데이터를 수집해볼 것이다.

크롤러 역할을 하는 함수를 각각 하나씩 만들어 놓고,
콘텐츠를 저장할 클래스도 하나 만들어 클래스에 함수를 투입하여 인스턴스를 찍어내는 방식으로 구조를 만들어보자.

이때, 콘텐츠를 저장할 클래스를 통해 우리는 크롤러의 데이터 수집 모델을 기획할 수 있다.
"""

import requests
import ssl
from bs4 import BeautifulSoup
from urllib.request import urlopen

context = ssl._create_unverified_context()

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
def getPage(url):
    req = urlopen(url, context = context)
    return BeautifulSoup(req, 'html.parser')

def scrapeNYTimes(url):
    bs = getPage(url)
    title = bs.find('h1').text        
    # 생소한 함수 .text 이다. 객체 지향적이라고 하는데, 사실 그냥 get_text()이다.
    # 차이점은 get_text() 는 괄호가 있기 때문에 여기에 매개변수를 지정해 줄 수 있다는 점이다.
    lines = bs.select('div.StoryBodyCompanionColumn div p')
    # select()는 말하자면 findAll() 이라고 생각하면 된다. find()에 대응되는 것은 참고로 select_one()이다.
    # 하지만 find계열과는 달리, css 문법에 따라 지정하는 것이라고 한다.더 직관적이라는 말도 있다.
    body = '\n'.join([line.text for line in lines])
    # 'something'.join(a) 함수는 a 이터레이터를 하나씩 뽑아 하나의 string을 만들되,
    # 앞의 something을 구분자로 하여 만드는 것이다.
    # 즉 여기서는, 뽑아낸 lines를 한줄씩 뽑아 중간중간 줄바꿈으로 구분해 주었다고 보면 된다.
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = getPage(url)
    title = bs.find('h1').text
    body = bs.find('div', {'class':'post-body post-body-enhanced'}).get_text()
    # 책에는 attributes 부분이, {'class', 'post-body'}로 되어 있는데, 
    # AttributeError가 난다. Nonetype이 나온다는 거지. 즉, 형식이 바뀐 것으로 보인다.
    # 그래서 다시 형식을 바꿔서 코드를 작성해주었다.
    # 어 그런데, 다시 해봐도 또 똑같길래, getPage(url)부분을 requests 모듈이 아니라 urlopen 모듈을
    # 사용하도록 수정해줬다. 그러니까 된다. 다시 책에 나온 attribute을 작성해줬다. 그래도 된다.
    # 그래도 그냥 내가 찾은 걸로 썼다. 필요하면 나중에 책을 확인해보자.
    return Content(url, title, body)

# 코드의 스크레이퍼 함수는 총 두 개로 구성되어 있다. 어떤 용도의 차이가 있는지 알아내보자.

url = """
https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/
"""
content = scrapeBrookings(url)
print('Title: {}'.format(content.title))
print('URL: {}\n'.format(content.url))
print(content.body)

url = """
https://www.nytimes.com/2018/01/25/opinion/sunday/silicon-valley-immortality.html
"""

content = scrapeNYTimes(url)
print('Title: {}'.format(content.title))
print('URL: {}\n'.format(content.url))
print('Content : \n{}', format(content.body))

"""
우선 출력 너무 잘되고, 이런식으로 앞으로도 객체지향으로 짜주면 될 것 같다.
다만, 이 코드를 전부 이해하기에는 css 선택자(문법)을 모른다. 그 select 함수 때문에 .join도 쓴 것인데,
이런 게 있구나 정도로만 알고 우선 지나가고 차차 css 선택자까지 알아보자. 
"""

Title: Delivering inclusive urban access: 3 uncomfortable truths
URL: 
https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/



The past few decades have been filled with a deep optimism about the role of cities and suburbs across the world. These engines of economic growth host a majority of world population, are major drivers of economic innovation, and have created pathways to opportunities for untold amounts of people.







Jeffrey Gutman

					Former Nonresident Fellow, Global Economy and Development										







Adie Tomer

					Senior Fellow - Brookings Metro 

 Twitter
AdieTomer





But all is not well within our so-called Urban Century. Rapid urbanization, rising gentrification, concentrated poverty, and shortages of basic infrastructure have combined to create spatial inequity in cities and suburbs across the globe. The challenges of housing, moving, and employing so many people have led to longer travel 

In [8]:
import requests
import ssl
from bs4 import BeautifulSoup
from urllib.request import urlopen

context = ssl._create_unverified_context()
        
def getPage(url):
    req = urlopen(url, context = context)
    return BeautifulSoup(req, 'html.parser')

bs = getPage('https://finance.naver.com/item/main.naver?code=035720')
grays = bs.find('div', {'id':'tab_con1'}).findAll('div', {'class':'gray'})
per = grays[-1].find('td').text.strip() # 공백이 너무 많아서 strip()을 하나 씌워줬다. 
print("'카카오'의 PER : ", per)

"""
뭔가 느낌을 좀 알 것 같아서,
궁극적인 GOAL대로 네이버 금융에서 per 값을 도출하는 작업을 한번 해봤다.

해보니까 사실 내가 필요한 것은,
종목마다 네이버 금융에서 도출하는 것이기 때문에 그다지 어렵지 않겠다는 생각이 들었다.
네이버 자체가 스크롤을 하기에도 매우 훌륭하고 구조가 확실한 사이트이기도 하니까 말이다.

전체적인 스크롤의 과정은, 재귀적으로 이리저리 왔다갔다하면서 모든 종목의 값들을 추출하고 저장하는 것이다.
'탐색' '추출' '저장' 의 과정이라고 한다면,
지금 한 것은, '추출'이다.
'탐색'도 이미 배워서 할 수도 있을 지 모른다.
6장에서 SQL에 '저장'하는 데 까지 우선 쭉 달려보자. 
"""

'카카오'의 PER :  6.16배


"\n뭔가 느낌을 좀 알 것 같아서,\n궁극적인 GOAL대로 네이버 금융에서 per 값을 도출하는 작업을 한번 해봤다.\n\n해보니까 사실 내가 필요한 것은,\n종목마다 네이버 금융에서 도출하는 것이기 때문에 그다지 어렵지 않겠다는 생각이 들었다.\n네이버 자체가 스크롤을 하기에도 매우 훌륭하고 구조가 확실한 사이트이기도 하니까 말이다.\n\n전체적인 스크롤의 과정은, 재귀적으로 이리저리 왔다갔다하면서 모든 종목의 값들을 추출하고 저장하는 것이다.\n'탐색' '추출' '저장' 의 과정이라고 한다면,\n지금 한 것은, '추출'이다.\n'탐색'도 이미 배워서 할 수도 있을 지 모른다.\n6장에서 SQL에 '저장'하는 데 까지 우선 쭉 달려보자. \n"

In [2]:
"""
자. 어제 유튜브를 통해 css 선택자를 대략 공부해왔다.
간단하게 정리/메모를 해놓고, 위의 스크레이퍼 함수가 어떤 뜻이었는 지까지 알아보자.

* : 모든 것을 지칭하는 선택자. 예를 들면 모든 html 파일을 노란색으로 칠하고 싶다던지 할 때 필요하다.
Tag : 태그의 이름. 예를 들면 div 가 있다. 참고로 div는 왼쪽부터 오른쪽의 한 층 전부를 차지한다고 한다.
#id : 태그의 attribute 중 id attribute value를 지칭.
.class : 태그의 attribute 중 class attribute value를 지칭. 즉, id와 class는 정석적인 attribute 이라는 것.
':' : 얘만 '' 처리를 해줬는데, 말그대로 : 여서 그렇다. 그냥 state를 의미하는 데, 버튼에 화살표가 올라간 state
등을 지칭한다. 웹 크롤링에서 실질적으로 사용할 일은 없을 것 같다.
[] : attribute을 지정하는 것인데, 아무래도 id나 class 이외의 속성과 속성값을 지정할 때 사용할 것이다.
예를 들면 다음과 같다. div[href^="naver"]. 이것은, div 태그 중 하이퍼 레퍼런스 속성 값이 naver로 시작하는 것을
지칭하는 것이다. 정규표현식과 함께 자주 사용되는 것 같다.

"""
# lines = bs.select('div.StoryBodyCompanionColumn div p')

# find() 함수에 대응시켜 지정해보자.
# bs.find('div',{'class' : 'StoryBodyCompanionColumn'}).find('div').findAll('p')

"""
자. 특정한 클래스 속성값을 가지고 있는 div 태그를 데려왔고,
그 안의 div 태그도 불러왔다.
그 안에는 많은 p 태그가 있을 것인데, 한번에 그것을 lines로 추출하고 
그 이후에 리스트 컴프리헨션과 .join() 함수를 이용해 다시 body로 만들었다.
"""

# 정말 혹~시나 하는 말이지만, 여기서 클래스는 당연히 두 가지 의미로 쓰였다.
# 파이썬의 class와, html의 attribute 일종인 class로 말이다.

'\n자. 특정한 클래스 속성값을 가지고 있는 div 태그를 데려왔고,\n그 안의 div 태그도 불러왔다.\n그 안에는 많은 p 태그가 있을 것인데, 한번에 그것을 lines로 추출하고 \n그 이후에 리스트 컴프리헨션과 .join() 함수를 이용해 다시 body로 만들었다.\n'

In [3]:
"""
여기서 사용한 '함수 내의' 변수들 중, 사이트마다 달라지는 변수는 
각 정보를 얻는 데에 사용한 css 선택자 뿐이다. (혹은 find를 위한 태그 문자열과 키/값 속성 딕셔너리)

따라서 더욱 편리한 사용을 위해, 사이트 구조와 데이터 위치를 정의ㅎㅏ는 매개변수만 바꾸어주면 되는데,
수집할 정보에 대응하는 css 선택자를 각각 문자열 하나로 만들고, 이들을 딕셔너리 객체에 모아보자.

"""

class Content:
    # 글/ 페이지 인스턴스를 형상하는 클래스
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
    def printing(self):
        print("URL : {}".format(self.url))
        print("TITLE : {}".format(self.title))
        print("BODY : \n{}".format(self.body))
        
class Website:
    # 웹사이트 구조 인스턴스를 형상하는 클래스
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
    
# 이 클래스들을 활용하여 주어진 웹 페이지에 존재하는 URL의 제목과 내용을 스크랩할 크롤러를 작성해봬자.

In [4]:
from bs4 import BeautifulSoup
import requests

class Crawler:
    """
    원래 클래스는 항상 __init__(self): 로 시작하는 줄 알았는데,
    반드시 그런 것만은 아닌가보다.
    
    다음 쉘에서처럼 parse를 호출하면서 필요한 매개변수들을 직접 넣어주는 것으로도 가능하다.
    
    이럴 경우의 장점은, 다음 쉘을 보면 알 수 있듯 여러개의 Crawler 객체를 만들지 않고도
    하나의 객체에 속한 내부 함수의 매개변수를 바꿔주는 것만으로 원하는 결과를 얻기에 충분하다는 것이다!
    """

    def getPage(self,url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException: # 에러가 생겼을 때에는 객체를 반환하지 말어라!
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        """
        BeautifulSoup 객체와 선택자를 받아 크롤링을 실행해주는 함수. 
        주어진 선택자에 따라, 제목을 크롤링 할 수도 기사내용을 크롤링 할 수도 있다.
        """
        
        selectedElems = pageObj.select(selector) # css 선택자를 이용해 원하는 요소들을 통째로 저장.
        if selectedElems is not None and len(selectedElems) > 0: # 그러니까 값이 있으면!
            return '\n'.join([elem.get_text() for elem in selectedElems]) # 위에서 했던 것과 같다.
        return ''
    
    def parse(self, site, url):
        """
        getPage로 url을 받아 safeGet으로 원하는 내용을 추출한 뒤, Content 인스턴트를 만들어 값을 저장한다.
        """
        
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, site.titleTag) # 이때 선택문으로는 받아낸 site 인스턴스를 이용한다.
            body = self.safeGet(bs, site.bodyTag) # 즉, site에는 Website 클래스의 객체를 넣어주어야 한다.
            if title != '' and body != '':
                content = Content(url, title, body)
                content.printing()

In [5]:
crawler = Crawler()

# 웹사이트 인스턴스 생성에 필요한 input 테이블 (어디까지 for css 선택자를 위한 것)
siteData = [
    ['O\'Reilly Media', 'http://oreilly.com','h1','section#product-description'],
    ['Reuters','http://reuters.com','h1','div.StandardArticleBody_body_1gnLA'],
    ['Brookings','http://www.brookigs.edu','h1','div.post-body']
]
# 웹사이트 인스턴스를 담을 테이블
websites = []
# bs 객체로 만들 url 정보
urls = [
    'http://shop.oreilly.com/product/0636920028154.do',
    'http://reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0',
    'https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/'
]
# siteData를 이용해 인스턴스를 만들고 websites 테이블에 append()
for row in siteData:
    websites.append(Website(row[0],row[1],row[2],row[3]))
    
# parse 함수 내에서 Content 클래스의 객체를 만드는 것 까지 다해주기 때문에, 
# 만들어진 객체에 대입만 해주면 된다!
crawler.parse(websites[0], urls[0])
crawler.parse(websites[1], urls[1])
crawler.parse(websites[2], urls[2])

URL : https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
TITLE : Idea to Retire: Old methods of policy education
Idea to Retire: Old methods of policy education
BODY : 

Public policy and public affairs schools aim to train competent creators and implementers of government policy. While drawing on the principles that gird our economic and political systems to provide a well-rounded education, like law schools and business schools, policy schools provide professional training. They are quite distinct from graduate programs in political science or economics which aim to train the next generation of academics. As professional training programs, they add value by imparting both the skills which are relevant to current employers, and skills which we know will be relevant as organizations and societies evolve. 
The relevance of the skills that policy programs impart to address problems of today and tomorrow bears further discussion. We are livi

In [ ]:
# 4.3 크롤러 구성

"""
지금까지 챕터 4를 정리해보자.

먼저, 크롤러의 콘텐츠 모델과 html을 추출할 위치/구조를 클래스로 형상하였다.
그 뒤 getPage 등 내부 함수들을 포함한 크롤러도 클래스로 형상하였고,
하나의 크롤러 객체만을 찍어내 여러 사이트들을 깨끗하게 추출하는 코드를 작성해보았다.

즉, 웹사이트 레이아웃을 객체 지향을 통해, 유연하고 수정하기 편하도록 만들어 준 것이다.

하지만, 아무리 그렇다고 해도 스크랩할 링크를 직접 하나씩 찾아 넣어야 한다면 별로 도움이 되지 않는다.
따라서 챕터 3에서 했던 내용과 같이, 웹사이트를 크롤링하고 새 페이지를 자동으로 찾는 방법을 적용해야 한다.

이제부터는 챕터 3에서 했던 내용과 이번 챕터에서 알아본 방법들을 어떻게 통합할지 알아볼 것이다.

우선, 기본 웹 크롤러 구조를 세 가지 알아보자. 
물론 실무에서는 경우에 따라 여기 저기 조금씩 수정할 필요는 있을 것이다.
"""

In [ ]:
# 4.3.1 검색을 통한 사이트 크롤링